In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv("C:/Users/Alisha/Desktop/Files/Google_Stock_Price_Train.csv")

In [3]:
train.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [4]:
train.shape

(1258, 6)

In [5]:
A=train.to_numpy()[:,1:2]

In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
A_scaled = sc.fit_transform(A)

xtrain=[]
ytrain=[]
for i in range(60,1258):
    xtrain.append(A_scaled[i-60:i,0])
    ytrain.append(A_scaled[i,0])

xtrain=np.array(xtrain)
ytrain=np.array(ytrain)

In [7]:
A_scaled.dtype

dtype('float64')

In [8]:
xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1], 1))

In [9]:
import tensorflow as tf

model=tf.keras.Sequential()

model.add(tf.keras.layers.LSTM(units=50,return_sequences=True,input_shape=(xtrain.shape[1],1)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.LSTM(units=50,return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.LSTM(units=50,return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(units=1))

model.compile(optimizer="adam",loss="mean_squared_error",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 50)            10400     
_________________________________________________________________
dropout (Dropout)            (None, 60, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 50)            0         
_________________________________________________________________
dense (Dense)                (None, 60, 1)             5

In [10]:
history=model.fit(xtrain,ytrain,epochs=10,batch_size=32)

Epoch 1/10
38/38 [==============================] - 10s 72ms/step - loss: 0.0747 - accuracy: 0.0015
Epoch 2/10
38/38 [==============================] - 2s 58ms/step - loss: 0.0259 - accuracy: 0.0016
Epoch 3/10
38/38 [==============================] - 2s 61ms/step - loss: 0.0204 - accuracy: 0.0016
Epoch 4/10
38/38 [==============================] - 2s 60ms/step - loss: 0.0174 - accuracy: 0.0016
Epoch 5/10
38/38 [==============================] - 2s 60ms/step - loss: 0.0154 - accuracy: 0.0016
Epoch 6/10
38/38 [==============================] - 2s 62ms/step - loss: 0.0140 - accuracy: 0.0016
Epoch 7/10
38/38 [==============================] - 2s 61ms/step - loss: 0.0131 - accuracy: 0.0016
Epoch 8/10
38/38 [==============================] - 2s 60ms/step - loss: 0.0121 - accuracy: 0.0016
Epoch 9/10
38/38 [==============================] - 2s 57ms/step - loss: 0.0115 - accuracy: 0.0017
Epoch 10/10
38/38 [==============================] - 2s 62ms/step - loss: 0.0109 - accuracy: 0.0016


In [12]:
B=pd.read_csv("C:/Users/Alisha/Desktop/Files/Google_Stock_Price_Test.csv")
B.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [13]:
real_stock_price=B.to_numpy()[:,1:2]

In [14]:
tot = pd.concat((train['Open'], B['Open']), axis = 0)
C = tot[len(tot) - len(B) - 60:].values
C = C.reshape(-1,1)
C = sc.transform(C)

In [15]:
xtest = []
ytest=[]
for i in range(60, 80):
    xtest.append(C[i-60:i, 0])
    ytest.append(C[i,0])
xtest,ytest = np.array(xtest),np.array(ytest)
xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
pred = model.predict(xtest)

In [16]:
pred=pred.reshape(20,60)
pred = sc.inverse_transform(pred)

In [19]:
xtest

array([[[0.9299055 ],
        [0.93113327],
        [0.92750577],
        ...,
        [0.95725128],
        [0.93796041],
        [0.93688146]],

       [[0.93113327],
        [0.92750577],
        [0.94415507],
        ...,
        [0.93796041],
        [0.93688146],
        [0.08581368]],

       [[0.92750577],
        [0.94415507],
        [0.93876032],
        ...,
        [0.93688146],
        [0.08581368],
        [0.09701243]],

       ...,

       [[0.96054394],
        [0.9371419 ],
        [0.92841729],
        ...,
        [0.02409033],
        [0.0159238 ],
        [0.01078949]],

       [[0.9371419 ],
        [0.92841729],
        [0.90804747],
        ...,
        [0.0159238 ],
        [0.01078949],
        [0.00967334]],

       [[0.92841729],
        [0.90804747],
        [0.8771858 ],
        ...,
        [0.01078949],
        [0.00967334],
        [0.01642607]]])

In [20]:
pred

array([[430.29416, 559.9009 , 675.93604, ..., 776.42596, 775.52856,
        774.1764 ],
       [430.36407, 559.7949 , 676.8871 , ..., 775.52844, 774.1763 ,
        730.8183 ],
       [430.1576 , 560.51337, 678.01605, ..., 774.1762 , 730.8182 ,
        668.30365],
       ...,
       [432.04605, 563.84906, 680.67755, ..., 318.6922 , 314.73273,
        309.7125 ],
       [430.70657, 560.5489 , 675.11505, ..., 314.732  , 309.7117 ,
        304.21887],
       [430.20944, 558.16034, 669.31665, ..., 309.71097, 304.21814,
        299.09412]], dtype=float32)

In [21]:
ytest

array([0.08581368, 0.09701243, 0.09433366, 0.09156187, 0.07984225,
       0.0643277 , 0.0585423 , 0.06568569, 0.06109085, 0.06639259,
       0.0614257 , 0.07474514, 0.02797827, 0.02379269, 0.02409033,
       0.0159238 , 0.01078949, 0.00967334, 0.01642607, 0.02100231])